In [1]:
from platform import python_version
# !pip install shap==0.39
# !pip install transformers==4.5
python_version()

'3.7.13'

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline
from Huggingface.utils import HuggingfaceDatasetManager, DatasetTypeEnum, explain_text, TransformersModelTypeEnum
import torch

In [4]:
torch.cuda.is_available()

True

## Load model from Huggingface

In [5]:
model_type = TransformersModelTypeEnum.GA_DISTIL_ROBERTA_BASE_FINETUNED_FAKE_NEWS
model_name = model_type.value['NAME']
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()
pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, binary_output=True, device=0)
embedding_dim = model.roberta.embeddings.word_embeddings.embedding_dim

## Load the respective dataset for the model

In [6]:
# dataset is unknown, we use the most common one
dataset_type = DatasetTypeEnum.ROBERTA_FAKE_NEWS
dataset_manager = HuggingfaceDatasetManager(dataset_type, embedding_dim=embedding_dim)

Loading instances from dir: /home/sersery/Desktop/TUM Informatik/SS22/Thesis/Code/Explainability_of_FND_Models/Huggingface/data/RobertaFakeNews/True.csv
Loaded 21417 instances from /home/sersery/Desktop/TUM Informatik/SS22/Thesis/Code/Explainability_of_FND_Models/Huggingface/data/RobertaFakeNews/True.csv
Loading instances from dir: /home/sersery/Desktop/TUM Informatik/SS22/Thesis/Code/Explainability_of_FND_Models/Huggingface/data/RobertaFakeNews/Fake.csv
Loaded 23481 instances from /home/sersery/Desktop/TUM Informatik/SS22/Thesis/Code/Explainability_of_FND_Models/Huggingface/data/RobertaFakeNews/Fake.csv


In [7]:
true_samples = dataset_manager.fetch_true_samples(sample_count=10)
true_sample_shap_values = explain_text(model_type=model_type, pipeline=pipeline, text=true_samples[0])

Getting the following indexes: [16566 16319 18924 10414  4207   275 21292 10726 11838 20754]
Explaining the following text: 
<--------------------------------------------------------------------------------------------->
NEW DELHI (Reuters) - India has agreed to hold talks on Afghanistan with the United States and Kabul, Indian Foreign Minister Sushma Swaraj said on Wednesday after meeting U.S. Secretary of State Rex Tillerson in New Delhi. Swaraj said she also discussed deepening economic ties with the United States. 
<--------------------------------------------------------------------------------------------->
[{'label': 'LABEL_0', 'score': 6.810428749304265e-05}, {'label': 'LABEL_1', 'score': 0.9999319314956665}]
Predicted Fake with score: 6.810428749304265e-05
Predicted True with score: 0.9999319314956665


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
